In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
from sklearn.model_selection import KFold

In [4]:
from tensorflow.keras.layers import Conv2D

In [5]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import concatenate

In [6]:
from tensorflow import keras

In [7]:
from tensorflow.keras.layers import Bidirectional

In [8]:
from tensorflow.keras.models import Model

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

In [11]:
from tensorflow.python.keras.layers.embeddings import Embedding

In [12]:
import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer

In [13]:
train_df=pd.read_csv('train.csv')

In [14]:
test_df=pd.read_csv('test.csv')

In [15]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [16]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower()
       
    ## Remove stop words (mejor las dejo)
    #text = text.split()
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]  
    #text = " ".join(text)

    ## Clean the text
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    
    ## Stemming
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)

    return text

In [17]:
train_df['text'] = train_df['text'].replace(r'http\S+', '', regex=True)
test_df['text'] = test_df['text'].replace(r'http\S+', '', regex=True)

In [18]:
train_df.text=train_df.text.map(lambda x: clean_text(x))
test_df.text=test_df.text.map(lambda x: clean_text(x))

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_data = pad_sequences(train_sequences, maxlen=180)

In [20]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [21]:
EMBEDDING_FILE = 'glove-dataset/glove.twitter.27B.200d.txt' 
#Por ahora lo dejoe n 50, supongo que solo mejora con 200 si complejizo la red nueronal
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, 'r', encoding='utf-8'))

In [22]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((50000, 200)) #50 o 200, segun las dimensiones del objeto que use
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [24]:
model_glove = Sequential()
model_glove.add(Embedding(50000, 200, input_length=180, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
#separo train y test
x_train, x_test, y_train, y_test = train_test_split(train_data, train_df['target'], test_size=0.2)

In [26]:
model_glove.fit(x_train, y_train, epochs = 2)

Epoch 1/2
191/191 [==============================] - 15s 78ms/step - loss: 0.4940 - accuracy: 0.7747
Epoch 2/2
191/191 [==============================] - 14s 71ms/step - loss: 0.4034 - accuracy: 0.8279


In [27]:
pred = model_glove.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [28]:
pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [29]:
f1_score(y_test, pred, average='macro')

0.7814285973804204

 # Calculo con k-folds

In [30]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [31]:
#Igual esta "mal", deberia vectorizar y entrenar solo los valores del train (creo)

In [33]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train,x_test = train_data[train_index], train_data[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    model_glove = Sequential()
    model_glove.add(Embedding(50000, 200, input_length=180, weights=[embedding_matrix], trainable=False))
    model_glove.add(Dropout(0.2))
    model_glove.add(Conv1D(64, 5, activation='relu'))
    model_glove.add(MaxPooling1D(pool_size=4))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model_glove.fit(x_train,y_train, epochs=2, batch_size=20)
    
    pred = model_glove.predict_classes(x_test)

    test_error.append(f1_score(y_test, pred, average='macro'))
    
print('Error de test:',test_error)

Epoch 1/2
305/305 [==============================] - 16s 53ms/step - loss: 0.4870 - accuracy: 0.7703
Epoch 2/2
305/305 [==============================] - 16s 54ms/step - loss: 0.4067 - accuracy: 0.8243
Epoch 1/2
305/305 [==============================] - 17s 55ms/step - loss: 0.4780 - accuracy: 0.7849
Epoch 2/2
305/305 [==============================] - 17s 55ms/step - loss: 0.3968 - accuracy: 0.8261
Epoch 1/2
305/305 [==============================] - 18s 59ms/step - loss: 0.4705 - accuracy: 0.7783
Epoch 2/2
305/305 [==============================] - 18s 59ms/step - loss: 0.3943 - accuracy: 0.8297
Epoch 1/2
305/305 [==============================] - 17s 57ms/step - loss: 0.4841 - accuracy: 0.7721
Epoch 2/2
305/305 [==============================] - 17s 55ms/step - loss: 0.4017 - accuracy: 0.8258
Epoch 1/2
305/305 [==============================] - 17s 55ms/step - loss: 0.4833 - accuracy: 0.7747
Epoch 2/2
305/305 [==============================] - 17s 56ms/step - loss: 0.4093 - accurac

# Realizo la prediccion

In [36]:
#Creo el modelo devuelta
model_glove = Sequential()
model_glove.add(Embedding(50000, 200, input_length=180, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(300))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model_glove.fit(train_data, train_df['target'], epochs = 2)

Epoch 1/2
238/238 [==============================] - 17s 72ms/step - loss: 0.4830 - accuracy: 0.7795
Epoch 2/2
238/238 [==============================] - 17s 72ms/step - loss: 0.4041 - accuracy: 0.8181


In [38]:
test_df['text'] = test_df['text'].map(lambda x: clean_text(x))

In [39]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [40]:
predictions= model_glove.predict_classes(test_data)

In [41]:
predictions

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [42]:
sample.target = predictions

In [43]:
sample.to_csv('neural_network_glove_2_epochs.csv', index=False)

In [44]:
sample.target.value_counts()

0    2134
1    1129
Name: target, dtype: int64

# Intento agregarle multiple inputs

In [45]:
from sklearn.preprocessing import MinMaxScaler

In [46]:
cs = MinMaxScaler()

In [71]:
train_input_2 = pd.read_csv("train_limpio_input_2_NN.csv",encoding = "ISO-8859-1")
test_input_2 = pd.read_csv("test_limpio_input_2_NN.csv",encoding = "ISO-8859-1")

In [72]:
#Le saco el target 
test_ids = test_input_2.pop("id")
target = train_input_2.target
train_input_2 = train_input_2.drop(columns=['target'])

In [73]:
test_input_2 = test_input_2.drop(columns=['text'])
train_input_2 = train_input_2.drop(columns=['text'])

In [74]:
trainContinuous = cs.fit_transform(train_input_2)
testContinuous = cs.fit_transform(test_input_2)

In [56]:
trainContinuous.shape

(7613, 2850)

# Contolo con k-folds

In [87]:
nlp_input = keras.Input(shape=(180,), name='nlp_input')
meta_input = keras.Input(shape=(2850,), name='meta_input')

In [88]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train_words,x_test_words = train_data[train_index], train_data[test_index]
    x_train_numerical,x_test_numerical = trainContinuous[train_index], trainContinuous[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_input)
    drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)
    conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
    mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)
    nlp_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb
    
    x = concatenate([nlp_out, meta_input])
    x = Dense(24, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    
    model = Model(inputs=[nlp_input , meta_input], outputs=[x])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    model.fit([x_train_words, x_train_numerical], y_train, batch_size=128, epochs=7)
    
    predictions= model.predict([x_test_words,x_test_numerical])
    
    bin_pred = []
    for i in predictions:
        if (i<0.5):
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    test_error.append(f1_score(y_test, bin_pred, average='macro'))
    
print('Error de test:',test_error)

Epoch 1/7
48/48 [==============================] - 15s 313ms/step - loss: 0.5197 - accuracy: 0.7521
Epoch 2/7
48/48 [==============================] - 15s 317ms/step - loss: 0.4174 - accuracy: 0.8164
Epoch 3/7
48/48 [==============================] - 15s 322ms/step - loss: 0.3924 - accuracy: 0.8299
Epoch 4/7
48/48 [==============================] - 15s 320ms/step - loss: 0.3720 - accuracy: 0.8429
Epoch 5/7
48/48 [==============================] - 16s 329ms/step - loss: 0.3542 - accuracy: 0.8501
Epoch 6/7
48/48 [==============================] - 16s 329ms/step - loss: 0.3294 - accuracy: 0.8589
Epoch 7/7
48/48 [==============================] - 15s 322ms/step - loss: 0.3051 - accuracy: 0.8741
Epoch 1/7
48/48 [==============================] - 16s 323ms/step - loss: 0.5190 - accuracy: 0.7501
Epoch 2/7
48/48 [==============================] - 16s 324ms/step - loss: 0.4175 - accuracy: 0.8167
Epoch 3/7
48/48 [==============================] - 16s 326ms/step - loss: 0.3878 - accuracy: 0.8304


In [ ]:
#Con dropout, conv1D, maxpooling y 3 epochs (estandar):
Error de test: [0.7888178425057502, 0.7914766253113024, 0.7840449223167527, 0.746530115508615, 0.7930096559227526]
    
#SIN dropout,ni conv1D,ni maxpooling y con 3 epochs (da un poquito mejor sin nada de lo que le agregue):
Error de test: [0.7907451626963822, 0.786468158944994, 0.7854915546911962, 0.7873662261730481, 0.8080302711793146]
    
#Con dropout, conv1D, maxpooling PERO con 5 epochs (mejoro un poquito tambien):
Error de test: [0.805207329733444, 0.7851134723292488, 0.7747011121720304, 0.7838792991204754, 0.793572611501342]
    
#Con dropout, conv1D, maxpooling y 3 epochs PERO con vectores de 200dims:
Error de test: [0.8013598421267523, 0.7831023324122048, 0.7737440449504573, 0.7866347906188544, 0.8026946909920577]

#Mezcla de todo, ya que con todo lo mejoró(tarda una banda):



In [ ]:
predictions= model.predict([test_data,testContinuous])

In [ ]:
len(predictions)

In [80]:
bin_pred = []
for i in predictions:
    if (i<0.5):
        bin_pred.append(0)
    else:
        bin_pred.append(1)

In [83]:
sample.target = bin_pred

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [84]:
sample.to_csv('neural_network_doble_input_triple_epochs_more_features.csv', index=False)

### Mismo pero meto conv2d

In [ ]:
for train_index,test_index in kf.split(train_data):
    x_train_words,x_test_words = train_data[train_index], train_data[test_index]
    x_train_numerical,x_test_numerical = trainContinuous[train_index], trainContinuous[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_input)
    drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)
    conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
    mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)
    nlp_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb
    
    x = concatenate([nlp_out, meta_input])
    x = Dense(24, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    
    model = Model(inputs=[nlp_input , meta_input], outputs=[x])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    model.fit([x_train_words, x_train_numerical], y_train, batch_size=128, epochs=5)
    
    predictions= model.predict([x_test_words,x_test_numerical])
    
    bin_pred = []
    for i in predictions:
        if (i<0.5):
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    test_error.append(f1_score(y_test, bin_pred, average='macro'))
    
print('Error de test:',test_error)